In [ ]:
R.home()

In [ ]:
baizer::pkglib(tidyverse, Biostrings, Peptides, baizer, yaml)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- snakemake@config
ab_scheme <- str_to_lower(config[['ext_numbering']])

In [ ]:
# read from igblast

In [ ]:
parse_IgBlast_shm <- function(igblast_fmt7) {
  alltxt <- readLines(igblast_fmt7)
  Ltxt <- baizer::split_vector(alltxt,
    grep("# IGBLASTN", alltxt),
    bounds = "[)"
  )
  Ltxt <- Ltxt[-1]

  parse_mut <- function(x) {
    contig_id <- str_subset(x, "# Query:") %>%
      str_replace("# Query: ", "")

    # VDJ, query id, subject id, % identity, alignment length,
    # mismatches, gap opens, gaps...
    vdj <- c(
      x %>% str_subset("^V\\t") %>% .[1],
      x %>% str_subset("^D\\t") %>% .[1],
      x %>% str_subset("^J\\t") %>% .[1]
    )
    vdj_name <- c("V", "D", "J")
    vdj_mut <- vdj %>%
      str_split("\t") %>%
      map_chr(~ if (all(is.na(.x))) NA_character_ else .x[6])

    # region, from, to, length, matches, mismatches, gaps, percent identity
    cdrfr <- x %>% str_subset("^[CDRF123]+-IMGT+")
    cdrfr_name <- cdrfr %>%
      str_split("\t") %>%
      map_chr(1) %>%
      str_replace("-IMGT", "") %>%
      str_replace(" \\(germline\\)", "")
    if (any(duplicated(cdrfr_name)) == TRUE) {
      stop(str_c("duplicated region names  ", contig_id))
    }
    cdrfr_mut <- cdrfr %>%
      str_split("\t") %>%
      map_chr(6)


    res <- tibble( # nolint
      sequence_id = contig_id,
      name = c(vdj_name, cdrfr_name),
      mut = c(vdj_mut, cdrfr_mut)
    ) %>%
      # fill NA as 0
      dplyr::mutate(mut = as.double(mut), mut = ifelse(is.na(mut), 0, mut)) %>%
      pivot_wider(names_from = "name", values_from = "mut")
  }

  res <- Ltxt %>% map_dfr(parse_mut)

  # rename
  res <- res %>% rename_at(
    str_subset(colnames(.), "^FR\\d"),
    ~ str_replace(.x, "FR", "FWR")
  )
  return(res)
}

In [ ]:
TBigblast_tsv <- read_tsv(snakemake@input[['igblast_tsv']])
TBshm <- parse_IgBlast_shm(snakemake@input[['igblast_txt']])

In [ ]:
# read from changeo
TBchangeo <- bind_rows(read_tsv(snakemake@input[['changeo']]), read_tsv(snakemake@input[['changeo_fail']]))

In [ ]:
# read from anarci

In [ ]:
ab_schema_region <- read_csv(snakemake@params[['ab_schema_region']])

In [ ]:
parse_ANARCI_aaseq <- function(x, remove_gap = TRUE, keep_number = FALSE, ab_schema_region) {
  aa_cols <- colnames(x) %>% str_subset("^\\d+")
  res <- x %>%
    dplyr::select(sequence_id = "Id", dplyr::all_of(aa_cols)) %>%
    # trans TRUE, FALSE to the real character T, F
    dplyr::mutate(
      dplyr::across(
        dplyr::where(is.logical),
        ~ case_when(
          .x == TRUE ~ "T", .x == FALSE ~ "F",
          is.na(.x) ~ "-", TRUE ~ as.character(.x)
        )
      )
    ) %>%
    # trans NA to -, to avoid NA in sequence
    dplyr::mutate(
      dplyr::across(dplyr::all_of(aa_cols), ~ ifelse(is.na(.x), "-", .x))
    )

  # region seq aa
  region <- res %>%
    pivot_longer(-"sequence_id", names_to = "numbering", values_to = "aa") %>%
    dplyr::mutate(numbering = factor(.data[["numbering"]], aa_cols)) %>%
    dplyr::mutate(site = str_extract(.data[["numbering"]], "\\d+") %>%
      as.double()) %>%
    dplyr::arrange(.data[["sequence_id"]], .data[["numbering"]])

  region <- region %>%
    left_join(ab_schema_region,
      by = dplyr::join_by(between(site, start, end, bounds = "[]"))
    ) %>%
    dplyr::filter(!is.na(region)) %>%
    dplyr::arrange(region)

  region <- region %>%
    group_split(.data[["sequence_id"]]) %>%
    map_dfr(~ dplyr::summarise(.x,
      aa = str_c(aa, collapse = ""),
      .by = c("sequence_id", "region")
    )) %>%
    pivot_wider(names_from = "region", values_from = "aa") %>%
    rename_at(
      dplyr::pull(ab_schema_region, "region") %>% as.character(),
      ~ str_c(.x, "_aa")
    )

  # V-domain seq aa
  res <- res %>% tidyr::unite("seq_align_aa", dplyr::all_of(aa_cols),
    sep = "", remove = !keep_number
  )

  res <- left_join(region, res, by = "sequence_id")

  if (remove_gap == TRUE) {
    res <- res %>% dplyr::mutate(dplyr::across(
      ends_with("_aa"),
      ~ str_replace_all(.x, "-", "")
    ))
  }

  return(res)
}

In [ ]:
TBext_anarci_H <- read_csv(snakemake@input[['ext_anarci_H']])
TBext_anarci_L <- read_csv(snakemake@input[['ext_anarci_KL']])

In [ ]:
TBext_anarci_seq <- bind_rows(
    parse_ANARCI_aaseq(TBext_anarci_H, 
                       ab_schema_region=filter(ab_schema_region, scheme==ab_scheme, chain=='H')),
    parse_ANARCI_aaseq(TBext_anarci_L, 
                       ab_schema_region=filter(ab_schema_region, scheme==ab_scheme, chain=='L'))
) %>% rename_at(str_subset(colnames(.), 'cdr\\d_aa'), ~str_c(ab_scheme, .x, sep='_'))

In [ ]:
# main and fine x_call
gene_cols <- str_subset(colnames(TBigblast_tsv), '_call$')

TBigblast_tsv <- TBigblast_tsv %>%
    # extract first fine gene to _fine cols
    mutate(across(all_of(gene_cols), ~reg_match(.x, '[^,]+'), .names='{.col}_fine')) %>%
    # extract first main gene 
    mutate(across(all_of(gene_cols), ~reg_match(.x, '[^\\*]+'))) %>%
    # rename
    rename_at(c(gene_cols, str_c(gene_cols, '_fine')), 
              ~str_replace_all(.x, '_call', '_gene'))

In [ ]:
# cdr length, the length of NA cdr will be NA
cdr_aa_cols <- str_subset(colnames(TBigblast_tsv), 'cdr\\d_aa')
TBigblast_tsv <- TBigblast_tsv %>%
    mutate(across(all_of(cdr_aa_cols), ~nchar(.x), .names='{.col}_len'))

In [ ]:
# rename shm columns

TBshm <- TBshm %>% rename_at(-1, ~str_to_lower(.x) %>% str_c('_mismatch_nt')) %>%
    mutate(v_domain_shm_nt = 
           sum(c(v_mismatch_nt, d_mismatch_nt, j_mismatch_nt), na.rm=TRUE), 
           .by=sequence_id)

In [ ]:
# cdr3 aa feature, the feature of NA cdr3 will be NA
TBaafeature <- TBigblast_tsv %>% select(sequence_id, cdr3_aa) %>% mutate(
    cdr3_charge=ifelse(!is.na(cdr3_aa), charge(seq=cdr3_aa, pH=7.4, pKscale='EMBOSS'), NA),
    cdr3_hydrophobicity=map_dbl(cdr3_aa, ~ifelse(!is.na(.x), hydrophobicity(seq=.x, scale='Eisenberg'), NA)),
    cdr3_boman=map_dbl(cdr3_aa, ~ifelse(!is.na(.x), boman(seq=.x), NA)),
    # if nchar of cdr3 <= 2, instaIndex will throw an error
    cdr3_instaindex = map_dbl(cdr3_aa, ~ifelse(!is.na(.x) & nchar(.x) > 2, instaIndex(seq=.x), NA))
) %>% select(-cdr3_aa)

In [ ]:
##################################
### select cols
##################################

In [ ]:
TBigblast_tsv %>% colnames

In [ ]:
# columns from igblast
TBigblast_sel <- TBigblast_tsv %>% select(
                    sequence_id, productive_igblast=productive, chain=locus, 
                    v_gene, d_gene, j_gene, v_gene_fine, d_gene_fine, j_gene_fine, 
                    cdr1_aa_len, cdr2_aa_len, cdr3_aa_len,
                    cdr1_nt=cdr1, cdr1_aa, cdr2_nt=cdr2, cdr2_aa, cdr3_nt=cdr3, cdr3_aa, np1_nt=np1, np2_nt=np2,
                    fwr1_nt=fwr1, fwr1_aa, fwr2_nt=fwr2, fwr2_aa, fwr3_nt=fwr3, fwr3_aa, fwr4_nt=fwr4, fwr4_aa,
                    v_start=v_sequence_start, v_end=v_sequence_end, 
                    d_start=d_sequence_start, d_end=d_sequence_end,
                    j_start=j_sequence_start, j_end=j_sequence_end,
                    seq_nt=sequence, seq_aa=sequence_aa, seq_align_nt=sequence_alignment, gm_align_nt=germline_alignment
        ) 

# columns from shm
TBshm_sel <- TBshm %>%
    relocate(sequence_id, v_domain_shm_nt, contains('cdr'), contains('fwr'))

# columns from changeo
TBchangeo_sel <- TBchangeo %>% select(sequence_id, clone_changeo=clone_id, seq_gm=germline_alignment_d_mask)

# columns from anarci
TBext_anarci_seq <- TBext_anarci_seq

In [ ]:
##################################
### join
##################################

In [ ]:
TBjoin <- TBigblast_sel %>% left_join(TBshm_sel, by='sequence_id') %>%
    relocate(all_of(colnames(TBshm_sel)[-1]), .after=cdr3_aa_len)

TBjoin <- TBjoin %>% left_join(TBchangeo_sel, by='sequence_id') %>%
     relocate(clone_changeo, .after=productive_igblast)

TBjoin <- TBjoin %>% left_join(TBext_anarci_seq, by='sequence_id') %>%
    relocate(matches('cdr\\d_aa'), .after=fwr4_aa) %>%
    relocate(seq_align_aa, .after=seq_align_nt)

TBjoin <- TBjoin %>% left_join(TBaafeature, by='sequence_id') %>%
    relocate(all_of(colnames(TBshm_sel)[-1]), .after=cdr3_aa)

In [ ]:
# calculate shm ratio
TBjoin <- TBjoin %>% mutate(v_domain_shm_ratio = round(v_domain_shm_nt/nchar(seq_align_nt), 4), 
                  .after=v_domain_shm_nt)

In [ ]:
##################################
### widen for HL
##################################

In [ ]:
TBjoin <- TBjoin %>% mutate(HL=case_when(chain=='IGH' ~ 'H', chain=='IGL' ~ 'L', chain=='IGK' ~ 'L')) %>%
    mutate(HL=factor(HL, c('H', 'L'))) %>% 
    # remove the contigs unknown
    filter(!is.na(HL))

In [ ]:
pattern <- config[['cell_pattern']]

In [ ]:
TBjoin <- TBjoin %>% mutate(cell=reg_match(sequence_id, pattern), .after=sequence_id)

In [ ]:
# contig numbers of a cell, and whether there is only an unique H or L
TBunique <- TBjoin %>% group_by(cell, HL) %>% summarise(contig_num = n(), unique = n() == 1) %>%
    ungroup

In [ ]:
# for multi-contigs, only keep the first one with most umis
TBjoin <- TBjoin %>% group_by(cell, HL) %>% dplyr::slice(1) %>% ungroup %>%
    arrange(HL, cell, sequence_id)

In [ ]:
TBjoin <- TBjoin %>% left_join(TBunique, by=c('cell', 'HL')) %>%
    relocate(contig_num, unique, .after=productive_igblast)

In [ ]:
# pivot
keep_col <- c('cell', 'HL')
TBwider <- TBjoin %>% pivot_wider(names_from='HL', values_from=-all_of(keep_col))

In [ ]:
# unique, productive, clone_cellranger, class
TBwider <- TBwider %>% 
    mutate(productive_igblast=productive_igblast_H & productive_igblast_L,
           unique = unique_H & unique_L, 
           # class=str_sub(c_gene_H, 1, 4) %>% str_replace('GH', 'g'),
           clone_changeo=str_glue('{clone_changeo_H}-{clone_changeo_L}'),
           .after=cell
          )
# if single contig
TBwider <- TBwider %>% mutate(
    productive_igblast=ifelse(is.na(seq_nt_H) | is.na(seq_nt_L), FALSE, productive_igblast),
    unique=ifelse(is.na(seq_nt_H) | is.na(seq_nt_L), FALSE, unique)
)

In [ ]:
##################################
### mutation target
##################################

In [ ]:
# Hchain mut
orfH <- parse_ANARCI_aaseq(
            read_csv(snakemake@input[['anarci_H']]), keep_number=TRUE,
            ab_schema_region=filter(ab_schema_region, scheme=='imgt', chain=='H')
        ) %>% select(id=sequence_id, matches('^\\d')) %>% c2r('id')
gmH <- parse_ANARCI_aaseq(
            read_csv(snakemake@input[['gm_anarci_H']]), keep_number=TRUE,
            ab_schema_region=filter(ab_schema_region, scheme=='imgt', chain=='H')
        ) %>% select(id=sequence_id, matches('^\\d')) %>% c2r('id')

common_ids <- intersect(rownames(orfH), rownames(gmH))
common_cols <- intersect(colnames(orfH), colnames(gmH))
diff_bool_H <- orfH[common_ids, common_cols] != gmH[common_ids, common_cols]

# if germline is -
diff_bool_H <- ifelse(gmH != '-', diff_bool_H, FALSE)
colnames(diff_bool_H) <- str_c('imgt_H', colnames(diff_bool_H))

In [ ]:
# Lchain mut
orfL <- parse_ANARCI_aaseq(
            read_csv(snakemake@input[['anarci_KL']]), keep_number=TRUE,
            ab_schema_region=filter(ab_schema_region, scheme=='imgt', chain=='L')
        ) %>% select(id=sequence_id, matches('^\\d')) %>% c2r('id')
gmL <- parse_ANARCI_aaseq(
            read_csv(snakemake@input[['gm_anarci_KL']]), keep_number=TRUE,
            ab_schema_region=filter(ab_schema_region, scheme=='imgt', chain=='L')
        ) %>% select(id=sequence_id, matches('^\\d')) %>% c2r('id')

common_ids <- intersect(rownames(orfL), rownames(gmL))
common_cols <- intersect(colnames(orfL), colnames(gmL))
diff_bool_L <- orfL[common_ids, common_cols] != gmL[common_ids, common_cols]

# if germline is -
diff_bool_L <- ifelse(gmL != '-', diff_bool_L, FALSE)
colnames(diff_bool_L) <- str_c('imgt_L', colnames(diff_bool_L))

In [ ]:
TBmut <- TBwider %>% select(cell, sequence_id_H, sequence_id_L) %>%
    left_join(r2c(as.data.frame(diff_bool_H), 'sequence_id_H'), by='sequence_id_H') %>%
    left_join(r2c(as.data.frame(diff_bool_L), 'sequence_id_L'), by='sequence_id_L') %>%
    select(-matches('^sequence_id'))
TBmut <- TBmut %>% mutate(batch = sample, .after=cell)

In [ ]:
TBwider <- TBwider %>% mutate(batch = sample, .after=cell)

In [ ]:
# write
dir.create(dirname(snakemake@output[['csv']]), recursive = TRUE)
write_excel_csv(TBmut, snakemake@output[['mut']])
write_excel_csv(TBwider, snakemake@output[['csv']])